In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as f
import torch.utils.data

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import time

In [3]:
## device confic DEFAUTL GPU
device = torch.device("cuda: 0" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Device:  cpu


In [4]:
## prepocessing
def read_images(path, num_img):
    array = np.zeros([num_img, 64*32])
    i = 0
    for img in os.listdir(path):
        img_path = path + "\\" + img
        img = Image.open(img_path, mode = "r")
        data = np.asarray(img, dtype = "uint8")
        data = data.flatten()
        array[i, :] = data
        i += 1
    array = torch.from_numpy(array)
    return array

In [5]:
## Train
train_neg_path = r"C:\Users\Dileemre\Desktop\jupyter\DATAAITEAM\LSIFIR\Classification\Train\neg"
num_train_neg_img = 43390
x_train_neg_tensor = read_images(train_neg_path, num_train_neg_img)
y_train_neg_tensor = torch.zeros(num_train_neg_img, dtype = torch.long)

train_pos_path = r"C:\Users\Dileemre\Desktop\jupyter\DATAAITEAM\LSIFIR\Classification\Train\pos"
num_train_pos_img = 10208
x_train_pos_tensor = read_images(train_pos_path, num_train_pos_img)
y_train_pos_tensor = torch.ones(num_train_pos_img, dtype = torch.long)

print("x_train_neg_tensor", x_train_neg_tensor.size(), "y_train_neg_tensor", y_train_neg_tensor.size(), "\n"
      "x_train_pos_tensor", x_train_pos_tensor.size(), "y_train_pos_tensor", y_train_pos_tensor.size())

## Concat train
x_train = torch.cat((x_train_neg_tensor, x_train_pos_tensor), 0)
y_train = torch.cat((y_train_neg_tensor, y_train_pos_tensor), 0)

print("x_train", x_train.size(), "y_train", y_train.size())

x_train_neg_tensor torch.Size([43390, 2048]) y_train_neg_tensor torch.Size([43390]) 
x_train_pos_tensor torch.Size([10208, 2048]) y_train_pos_tensor torch.Size([10208])
x_train torch.Size([53598, 2048]) y_train torch.Size([53598])


In [6]:
## Test
test_neg_path = r"C:\Users\Dileemre\Desktop\jupyter\DATAAITEAM\LSIFIR\Classification\Test\neg"
num_test_neg_img = 22050
x_test_neg_tensor = read_images(test_neg_path, num_test_neg_img)
y_test_neg_tensor = torch.zeros(num_test_neg_img, dtype = torch.long)

test_pos_path = r"C:\Users\Dileemre\Desktop\jupyter\DATAAITEAM\LSIFIR\Classification\Test\pos"
num_test_pos_img = 5944
x_test_pos_tensor = read_images(test_pos_path, num_test_pos_img)
y_test_pos_tensor = torch.ones(num_test_pos_img, dtype = torch.long)

print("x_test_neg_tensor", x_test_neg_tensor.size(), "y_test_neg_tensor", y_test_neg_tensor.size(), "\n"
      "x_test_pos_tensor", x_test_pos_tensor.size(), "y_test_pos_tensor", y_test_pos_tensor.size())

## Concat test
x_test = torch.cat((x_test_neg_tensor, x_test_pos_tensor), 0)
y_test = torch.cat((y_test_neg_tensor, y_test_pos_tensor), 0)

print("x_test", x_test.size(), "y_test", y_test.size())

x_test_neg_tensor torch.Size([22050, 2048]) y_test_neg_tensor torch.Size([22050]) 
x_test_pos_tensor torch.Size([5944, 2048]) y_test_pos_tensor torch.Size([5944])
x_test torch.Size([27994, 2048]) y_test torch.Size([27994])


In [ ]:
## visualize
plt.imshow(x_train[45001, :].reshape(64, 32), cmap = "gray")

In [ ]:
## CNN

## Hyperparameter
num_epochs = 100
num_classes = 2
batch_size = 32
learning_rate = 0.0001

train = torch.utils.data.TensorDataset(x_train, y_train)
trainloader = torch.utils.data.DataLoader(train, batc_size = batch_size, shuffle = True)

test = torch.utils.data.TensorDataset(x_test, y_test)
testloader = torch.utils.data.DataLoader(test, batc_size = batch_size, shuffle = False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(10, 16, 5)
        self.fc1 = nn.Linear(16*13*5, 520)
        self.fc2 = nn.Linear(520, 130)
        self.fc3 = nn.Linear(130, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*13*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

In [ ]:
net = Net()
## net = Net().to(device)

criterion = nn.CrossEntropyLoss()

import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum = 0.8)

In [ ]:
start = time.time()

train_acc = []
test_acc = []
loss = []

use_gpu = False

for epoch in range(num_epochs):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data 
        inputs = input.view(batch_size, 1, 64, 32)
        inputs = inputs.float()
        
        if use_gpu:
            if torch.cuda.is_available():
                inputs, labels = inputs.to(device), labels.to(device)
       
        #zero gradient
        optimizer.zero_grad()
        
        #forward
        outputs = net(inputs)
        
        #loss
        loss = criterion(outputs, labels)
        
        #back
        loss.backward()
        
        #update weights
        optimizer.step()
        
    #test
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images.view(batch_size, 1, 64, 32)
            images = images.float()
            
            if use_gpu:
                if torch.cuda.is_available():
                    images, labels = images.to(device), labels.to(device)
           
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc1 = 100*correct/total
    print("accuracy test: ", acc1)
    test_acc.append(acc1)
    
    #train
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images.view(batch_size, 1, 64, 32)
            images = images.float()
            
            if use_gpu:
                if torch.cuda.is_available():
                    images, labels = images.to(devices), labels.to(devices)
            
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        acc2 = 100*correct/total
        print("accuracy train: ", acc2)
        train_acc.append(acc2)
        
print("Train is done.")
            
end = time.time()
process_time = (end - start)/60
print("Process Time: ", process_time)

In [ ]:
## RezNet

def conv3x3(in_planes, out_planes, stride = 1):
    
    